In [1]:
import sqlite3
import numpy as np

In [2]:
def print_schema(connection):
    for row in connection.execute("select sql from sqlite_master").fetchall():
        print(row[0])
        
def print_row(row):
    if row is None:
        print('NOT FOUND')
    else:
        for key in row.keys():
            print(f'{key}={row[key]}' )

In [3]:
oms_connection = sqlite3.connect('/store/scratch/dqm/OMS/runs_352322_358185.sql')
oms_connection.row_factory = sqlite3.Row
oms = oms_connection.cursor()
print_schema(oms_connection)

CREATE TABLE "runs" (
"index" INTEGER,
  "l1_hlt_mode_stripped" TEXT,
  "l1_key" TEXT,
  "duration" REAL,
  "b_field" REAL,
  "tier0_transfer" INTEGER,
  "l1_triggers_counter" REAL,
  "hlt_physics_throughput" REAL,
  "init_lumi" REAL,
  "last_update" TIMESTAMP,
  "beamactive_total_deadtime" REAL,
  "recorded_lumi" REAL,
  "delivered_lumi" REAL,
  "cmssw_version" TEXT,
  "crossing_angle_stop" REAL,
  "energy" REAL,
  "end_lumi" REAL,
  "hlt_physics_rate" REAL,
  "fill_number" REAL,
  "l1_hlt_mode" TEXT,
  "end_time" TIMESTAMP,
  "trigger_mode" TEXT,
  "l1_key_stripped" TEXT,
  "crossing_angle_start" REAL,
  "fill_type_party2" TEXT,
  "fill_type_party1" TEXT,
  "beta_star_start" REAL,
  "initial_prescale_index" REAL,
  "sequence" TEXT,
  "start_time" TIMESTAMP,
  "hlt_physics_size" REAL,
  "fill_type_runtime" TEXT,
  "hlt_key" TEXT,
  "clock_type" TEXT,
  "last_lumisection_number" REAL,
  "l1_rate" REAL,
  "l1_menu" TEXT,
  "run_number" INTEGER,
  "stable_beam" INTEGER,
  "beta_star_stop

In [4]:
query = """
SELECT
    run_number,
    start_time,
    end_time,
    duration,
    GEM,
    CSC,
    DQM,
    tier0_transfer
FROM
    runs
WHERE
    run_number = ?
"""

run_number = 357606
run_row = oms.execute(query, (run_number, )).fetchone()
print_row(run_row)

run_number=357606
start_time=2022-08-17 00:30:05+00:00
end_time=2022-08-17 02:49:56+00:00
duration=8391.0
GEM=1
CSC=1
DQM=1
tier0_transfer=1


In [5]:
hv_db = '/store/scratch/dqm/P5GEMOfflineMonitor/P5_GEM_HV_monitor_UTC_start_2022-04-25_16-07-57_end_2022-08-25_15-22-38.sql'
hv_connection = sqlite3.connect(hv_db)
hv_connection.row_factory = sqlite3.Row
hv_cursor = hv_connection.cursor()

print_schema(hv_connection)

CREATE TABLE "hv" (
"index" INTEGER,
  "region" INTEGER,
  "station" INTEGER,
  "layer" INTEGER,
  "chamber" INTEGER,
  "time" TIMESTAMP,
  "hv" REAL
)
CREATE INDEX "ix_hv_index"ON "hv" ("index")


In [6]:
hv_query = """
SELECT
    hv
FROM
    hv
WHERE
    time >= ?
    AND time <= ?
    AND region = ?
    AND station = ?
    AND chamber = ?
"""

region = 1
station = 1
chamber = 1
hv_query_parameters = (
    run_row['start_time'],
    run_row['end_time'],
    1, # region
    1, # station
    1, # chamber
)


hv_row_list = hv_cursor.execute(hv_query, hv_query_parameters).fetchall()
if hv_row_list is None:
    print('HV NOT FOUND')
else:
    print(f'found {len(hv_row_list)} records')

found 27 records


In [7]:
hv_list = [row['hv'] for row in hv_row_list]
hv_avg = np.mean(hv_list)

In [8]:
print(f'{run_number=:d}: {hv_avg=:.1f}')

run_number=357606: hv_avg=731.9
